# Faiss

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms supporting code for evaluation and parameter tuning

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader('../3.2_DataIngestion/speech.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200,chunk_overlap=30)
doc = text_splitter.split_documents(documents)

c:\Users\chara\OneDrive\Desktop\Krish_naik\LangChain\genv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 668, which is longer than the specified 200
Created a chunk of size 982, which is longer than the specified 200
Created a chunk of size 789, which is longer than the specified 200


In [3]:
doc

[Document(metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metad

In [4]:
embeddings = OllamaEmbeddings(model="gemma:2b")
db = FAISS.from_documents(doc, embeddings)
db

C:\Users\chara\AppData\Local\Temp\ipykernel_21648\2392033657.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:2b")


In [5]:
## querying
query = "what does the speaker believe is the main reason the united states should enter the war?"
docs = db.similarity_search(query)
docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

# As a Retriever

We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [6]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

# Similarity Search with score

There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 Distance. therefore, a lower score is better.

In [7]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='f4a5f7fc-9a22-48b9-af5b-dc64cc4e8a33', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  np.float32(3388.886)),
 (Document(id='96ed6ea4-6374-458e-8e4a-c03a9373f3f7', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is runnin

In [8]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-0.16215400397777557,
 1.36707603931427,
 0.4150804281234741,
 0.8524881601333618,
 0.31266430020332336,
 0.702712893486023,
 0.6999270915985107,
 0.22609680891036987,
 0.22628222405910492,
 0.15380799770355225,
 -1.3263243436813354,
 -0.21758051216602325,
 -0.4295211136341095,
 1.2290887832641602,
 1.39937424659729,
 -0.44809120893478394,
 1.9391682147979736,
 1.4046030044555664,
 1.6104315519332886,
 0.8547563552856445,
 0.7506499886512756,
 -0.7381850481033325,
 1.5525962114334106,
 0.5974863767623901,
 -0.5778413414955139,
 -0.8703343272209167,
 -1.713077187538147,
 -1.74373197555542,
 -1.1547952890396118,
 -2.0145323276519775,
 2.7708168029785156,
 -0.4651239216327667,
 1.3827522993087769,
 -0.4663841426372528,
 -0.7654770016670227,
 -0.5345679521560669,
 1.0569546222686768,
 -0.7500040531158447,
 -0.10110826045274734,
 -0.42914336919784546,
 -0.5099088549613953,
 -0.6356607675552368,
 0.6384579539299011,
 -0.36552828550338745,
 -1.2360914945602417,
 0.9784937500953674,
 0.380909

In [9]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='f4a5f7fc-9a22-48b9-af5b-dc64cc4e8a33', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='96ed6ea4-6374-458e-8e4a-c03a9373f3f7', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say

In [10]:
## Saving and Loading

db.save_local('faiss_index')

In [11]:
new_db = FAISS.load_local('faiss_index',embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

In [12]:
docs

[Document(id='f4a5f7fc-9a22-48b9-af5b-dc64cc4e8a33', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='96ed6ea4-6374-458e-8e4a-c03a9373f3f7', metadata={'source': '../3.2_DataIngestion/speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say